In [4]:
import ollama
import chromadb

documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
  
]

client = chromadb.Client()
collection = client.get_or_create_collection(name="docs")

# store each document in a vector embedding database
for i, d in enumerate(documents):
  response = ollama.embeddings(model="llama3", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[str(i)],
    embeddings=[embedding],
    documents=[d]
  )

In [18]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.document import Document

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
# text = "Moody Good is a British DJ and producer who has been active since 2013. He is known for his work in the dubstep and drum and bass genres. He has released music on labels such as Never Say Die Records, OWSLA, and Mau5trap. Some of his most popular tracks include 'Hotplate,' 'Super Lager,' and 'Musicbx.' He has also collaborated with artists such as Skrillex, Noisia, and Kill the Noise. In addition to his solo work, Moody Good is also one half of the duo 16bit."
text = "Here is the information about my dogs. Gracie May - Medium tan terrier, the absolute best."


docs = [Document(page_content=x) for x in text_splitter.split_text(text)]


from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_community import embeddings
from langchain_community.embeddings import OllamaEmbeddings
persist_directory = "/tmp/chromadb"
vectorstore = Chroma.from_documents(
    documents=docs,
    collection_name="freshh",
    embedding=OllamaEmbeddings(model='llama3')
)
retriever = vectorstore.as_retriever()


from langchain_community.llms import Ollama
llm = Ollama(model="llama3")


from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
template = """Respond to users as a chatbot.:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print(rag_chain.invoke("who is my dog?"))

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


I'm afraid I don't have any information about your dog! But if you'd like to know more about Moody Good, the British DJ and producer, I can certainly help with that. According to what I've learned, Moody Good has been active since 2013, producing music in the dubstep and drum and bass genres. He's released tracks on labels like Never Say Die Records, OWSLA, and Mau5trap, and has collaborated with artists like Skrillex, Noisia, and Kill the Noise. Plus, he's also half of the duo 16bit! Would you like to know more about his music or collaborations?
